# Test a trained model for [CIFAR-10](https://www.cs.toronto.edu/~kriz/cifar.html) dataset by [CIFAR-10.1](https://github.com/modestyachts/CIFAR-10.1) dataset
- Copy and append the following three cells [1] [2] and [3] to the cell where your model has trained, e.g., at the end of your Jupyter notebook.

In [ ]:
# [1]
!git clone https://github.com/modestyachts/CIFAR-10.1.git

## Make a data loader `cifar101loader` in PyTorch
- `transforms.Normalize` should be the same as that for the training dataset

In [ ]:
# [2]
import torch
from torch.utils.data import Dataset
import torchvision.transforms as transforms
import numpy as np

class CIFAR101Dataset(Dataset):

    def __init__(self, transform=None):
        self.data = np.load("./CIFAR-10.1/datasets/cifar10.1_v6_data.npy")
        self.labels = np.load("./CIFAR-10.1/datasets/cifar10.1_v6_labels.npy")
        self.transform = transform

    def __getitem__(self, index):
        if self.transform is not None:
            return self.transform(self.data[index]), self.labels[index]
        return self.data[index], self.labels[index]

    def __len__(self):
        return self.data.shape[0]

test_transform = transforms.Compose(
    [transforms.ToTensor(),
     transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))])
# !!! The same normalization as to the training dataset should be applied. !!!

batch_size = 32  # set as you like
cifar101dataset = CIFAR101Dataset(transform=test_transform)
cifar101loader = torch.utils.data.DataLoader(cifar101dataset, batch_size=batch_size, shuffle=False)

## Test your 'model'

In [ ]:
# [3]
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# assumes there exists your `model` that has already been trained.
model.eval()

n_samples = 0
preds_all = []
labels_all = []
for inputs, labels in cifar101loader:
    inputs = inputs.to(device)
    labels = labels.to(device)

    # track history if only in train
    with torch.set_grad_enabled(False):
        outputs = model(inputs)

    _, preds = torch.max(outputs, 1)
    preds_all += preds.tolist()
    labels_all += labels.tolist()
print(preds_all)
print(labels_all)


from sklearn.metrics import confusion_matrix, classification_report
print(classification_report(labels_all, preds_all))
cm = confusion_matrix(labels_all, preds_all)
print(cm)


classes = ('plane', 'car', 'bird', 'cat',
           'deer', 'dog', 'frog', 'horse', 'ship', 'truck')

import pandas as pd
cm_df = pd.DataFrame(cm, classes, classes)

import matplotlib.pyplot as plt
plt.figure(figsize=(12,12))

import seaborn as sns
sns.heatmap(cm_df, annot=True, fmt=".0f")

plt.xlabel('Predicted')
plt.xticks(rotation=45)
plt.ylabel('True')
plt.ioff()